In [1]:
#import packages
import os
import gzip
import json
import pickle
from typing import Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from PIL import Image
import torch
from config import config
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
#import train function
from akrmap import train_parametric_akr_map_model
from akrmap.akr_model import NeuralMapper

In [3]:
#import 
import numpy as np
txt_features0=np.load("data/HPSv2_all_train_txt.npy")
txt_features1=np.load("data/HPSv2_all_train_img.npy")
txt_features=torch.Tensor(np.concatenate([txt_features0,txt_features1],axis=1)).to("cuda")[:430000]
scores=torch.Tensor(100*np.sum(txt_features0*txt_features1, axis=1)).to("cuda")[:430000]
scores=scores.reshape((scores.shape[0],))
print(txt_features.shape)
print(scores.shape)

torch.Size([430000, 2048])
torch.Size([430000])


In [4]:
#train projection models
dim_input=txt_features.shape[1]
model_path, a1, a2, b=train_parametric_akr_map_model(txt_features, dim_input, config, scores)

100%|████████████████████████████████████████████████████████████████████████████████| 430/430 [02:11<00:00,  3.26it/s]

====> Epoch: 1. Time 0:02:11.884414. Average loss: 1.8743
====> Epoch: 1. Time 0:02:11.884414. Average Map loss: 1.9220


a1: 0.9992773532867432
a2: 1.5346351861953735
b: 1.446234107017517


100%|████████████████████████████████████████████████████████████████████████████████| 430/430 [02:07<00:00,  3.37it/s]

====> Epoch: 2. Time 0:02:07.716541. Average loss: 1.6746
====> Epoch: 2. Time 0:02:07.716541. Average Map loss: 1.4618
a1: 0.9991363883018494
a2: 1.8417285680770874
b: 1.530693769454956
Loss history saved in akrmaptsne_oqimai_epoch_2_loss.npy
final kernel params
a1: 0.9991363883018494
a2: 1.8417285680770874
b: 1.530693769454956
Training time: 0:04:19.604466


In [11]:
#load pretrained model
a1= 0.9987241625785828
a2= 7.954826354980469
b= 1.1770744323730469
model_path="saved_models/Text-to-image/HPSv2_HPD"
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights=model_path+".pt"
loaded_model = torch.load(model_weights, weights_only=False).to(dev)
bs = config.training_params["batch_size"]

In [15]:
#sampling function for fast kernel estimate
import random

def kernel_sample_and_get_remaining_ids(N, n, seed=None):
    # Ensure n is not larger than N
    if n > N:
        raise ValueError("n cannot be larger than N")
    
    # Set the random seed if provided
    if seed is not None:
        random.seed(seed)
    
    sampled_ids = random.sample(range(N), n)
    sampled_set = set(sampled_ids)
    return sampled_ids

In [12]:
#project embeddings
from torch.utils.data import TensorDataset, Dataset, DataLoader
labels_train=torch.tensor(np.array([1 for i in range(len(txt_features))])).to("cuda")
print(labels_train.shape)
labels_train=labels_train.reshape((labels_train.shape[0],1))
print(labels_train.shape)
points_train_ds = TensorDataset(txt_features, labels_train)
def get_batch_embeddings(pretrained_model: torch.nn.Module,
                         input_points: Dataset,
                         batch_size: int):
    """
    Yields final embeddings for every batch in dataset
    """
    pretrained_model.eval()
    test_dl = DataLoader(input_points, batch_size=batch_size, shuffle=False)
    for batch_points, batch_labels in test_dl:
        with torch.no_grad():
            embeddings = pretrained_model(batch_points)
            yield embeddings, batch_labels

pretrained_model=loaded_model
input_points_train=points_train_ds
batch_size=bs
pos_list=[]
for embeddings, batch_labels in get_batch_embeddings(pretrained_model,
                                                     input_points_train,
                                                     batch_size):
    x1 = embeddings[:, 0].detach().cpu().numpy()
    x2 = embeddings[:, 1].detach().cpu().numpy()
    pos_list.append(embeddings.detach().cpu().numpy())

projected_pos=np.concatenate(pos_list)
print("projected_pos", projected_pos.shape)


torch.Size([430000])
torch.Size([430000, 1])
projected_pos (430000, 2)


In [13]:
#define kernel regression function
def compute_kde_regression_new(X, Y, z_arr, a1, a2, b, h):
    n, d= X.shape
    a1 = torch.tensor(a1, dtype=torch.float32, device=dev)
    a2 = torch.tensor(a2, dtype=torch.float32, device=dev)
    b = torch.tensor(b, dtype=torch.float32, device=dev)
    X = torch.tensor(X, dtype=torch.float32, device=dev)
    Y = torch.tensor(Y, dtype=torch.float32, device=dev)
    dists = torch.cdist(X, Y)
    ra2=a2**2
    ra1=a1**2
    rb=b**2
    kernels=1/(1+ra2*dists**(2*rb))

    w_kde=z_arr @ (kernels)
    kde=torch.sum(kernels, axis=0)
    kde_reg=w_kde/kde
    return (ra1*kde_reg).detach().cpu().numpy()

def compute_kde_regression_baseline(X, Y, z_arr, a1=1, a2=1, b=1, h=0.9):
    n, d= X.shape
    a1 = torch.tensor(a1, dtype=torch.float32, device="cuda")
    a2 = torch.tensor(a2, dtype=torch.float32, device="cuda")
    b = torch.tensor(b, dtype=torch.float32, device="cuda")
    X = torch.tensor(X, dtype=torch.float32, device="cuda")
    Y = torch.tensor(Y, dtype=torch.float32, device="cuda")

    dists = torch.cdist(X, Y)
    ra2=1
    ra1=1
    rb=1
    w_kde=z_arr @ (torch.exp(-dists**(2)/((2 * h ** 2))))
    kde=torch.sum(torch.exp(-dists**(2)/((2 * h ** 2))), axis=0)
    kde_reg=w_kde/(kde+1e-20)
    return (kde_reg).cpu().numpy()

In [17]:
# compute contour without border
x_min, x_max=projected_pos.min(0), projected_pos.max(0)
# grid=2000
grid=50
xi = np.linspace(x_min[0], x_max[0], grid)
yi = np.linspace(x_min[1], x_max[1], grid)
z_list=[]
k_ids=kernel_sample_and_get_remaining_ids(len(txt_features),20000,seed=42)
for i in range(len(xi)):
    Y1=[]
    for j in range(len(yi)):
        Y1.append(np.array([xi[i],yi[j]]).reshape((1,2)))
    Y1_array=np.concatenate(Y1, axis=0)
    kde_y=compute_kde_regression_new(projected_pos[k_ids], Y1_array, scores[k_ids], a1, a2, b, 0.9)
    z_list.append(kde_y.reshape((len(yi), 1)))
  # print(kde_y.shape)
zi=np.concatenate(z_list, axis=1)
print(zi.shape)
print(np.min(zi), np.max(zi))

(50, 50)
21.3487 30.078835


In [18]:
# compute contour with border
from scipy.spatial import cKDTree
# Define grid bounds from projected positions
x_min, x_max = projected_pos.min(0), projected_pos.max(0)
# x_min= x_min-0.05*(x_max-x_min)
# x_max= x_max+0.05*(x_max-x_min)
# Define grid resolution
grid =1024
xi = np.linspace(x_min[0], x_max[0], grid)
yi = np.linspace(x_min[1], x_max[1], grid)

# Compute distances of grid points to projected_pos
X_grid, Y_grid = np.meshgrid(xi, yi)
grid_points = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T

# Construct k-d tree and find nearest distances
tree = cKDTree(projected_pos)
distances, _ = tree.query(grid_points)

# Reshape distances back to grid shape
distance_grid = distances.reshape(grid, grid)

# Set threshold for masking distant grid points
threshold = 0.1*(x_max[0]-x_min[0])  # Adjust as needed

# Compute KDE regression with distance-based masking
z_list = []
for i in range(len(xi)):
    Y1 = np.array([[xi[i], y] for y in yi])  # Shape (grid, 2)
    kde_y = compute_kde_regression_new(
        projected_pos[k_ids], Y1, scores[k_ids], a1, a2, b, 0.9
    )
    z_list.append(kde_y.reshape((len(yi), 1)))

zi = np.concatenate(z_list, axis=1)

# Apply NaN mask
zi[distance_grid > threshold] = np.nan


In [19]:
#display contour map
fig, ax1 = plt.subplots(nrows=1, figsize=(9, 7))
# Plot contour plot
cntr1 = ax1.contourf(xi, yi, zi, levels=40, cmap='RdYlBu_r')
fig.colorbar(cntr1, ax=ax1)
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Assume xi, yi, zi arrays are defined as in your original code
# xi, yi are the grid coordinates, and zi is the contour data (same as cntr1 in Matplotlib)

k_ids1=kernel_sample_and_get_remaining_ids(430000,2000,seed=42)
df = pd.DataFrame(projected_pos[k_ids1], columns=['Dimension 1', 'Dimension 2'])
df['Target'] = scores[k_ids1].detach().cpu().numpy()  # Add the target array to the DataFrame
df['Id'] = k_ids1
# Create the figure
fig = go.Figure()

# Add the contour plot (underlay)
# fig.add_trace(go.Contour(
#     z=zi,
#     x=xi,  # x-coordinates of the grid
#     y=yi,  # y-coordinates of the grid
#     colorscale="GnBu",      # Colormap
#     contours=dict(start=zi.min(), end=zi.max(), size=(zi.max() - zi.min()) / 60),  # Match levels (60 in original code)
#     showscale=False,        # Disable color bar for the contour layer
#     opacity=0.7             # Make it partially transparent
# ))
fig.add_trace(go.Heatmap(
    z=zi,
    x=xi,
    y=yi,
    colorscale='RdYlBu_r',  # Colormap
    showscale=False,    # Remove the color scale bar for the underlay (optional)
    zsmooth='best'      # Smooth interpolation for the heatmap
))

hover_text = [
    f"Dimension 1: {x:.2f}<br>Dimension 2: {y:.2f}<br>Target: {t:.2f}<br>Id: {u:.2f}"
    for x, y, t, u in zip(df['Dimension 1'], df['Dimension 2'], df['Target'], df['Id'])
]
# Add the scatter plot (overlay)
fig.add_trace(go.Scatter(
    x=df['Dimension 1'],
    y=df['Dimension 2'],
    mode='markers',
    marker=dict(
        size=20,                # Match point size
        color=df['Target'],     # Color based on the target column
        colorscale='RdYlBu_r',      # Match the colormap
        line=dict(width=0.5, color='white'),  # White edges around each point
        opacity=0.8             # Match transparency
    ),
    text=hover_text,  # Custom text for tooltips
    hoverinfo='text',  # Use only the custom text in the tooltip
))

# Update layout for style consistency
fig.update_layout(
    title="Interactive Scatter Plot with Contour Underlay",  # Add a title
    xaxis_title="Dimension 1",                               # X-axis label
    yaxis_title="Dimension 2",                               # Y-axis label
    autosize=False,                                          # Disable automatic resizing
    # width=900,                                               # Set figure width
    # height=700,                                              # Set figure height
    width=900,
    height=900,  # Set figure size to be square
    xaxis=dict(
        scaleanchor="y"  # Link x-axis scale to y-axis
    ),
    # xaxis=dict(scaleanchor='y'),                             # Fix aspect ratio 1:1
    paper_bgcolor='white',                                   # Background color
    plot_bgcolor='white',                                    # Plot area background
    font=dict(size=12, color='black'),                       # Font settings
    margin=dict(l=50, r=50, t=50, b=50),                     # Margins
)


# Show the plot
fig.show()